# パイプラインの要素を作成する

このNotebookは、[TUTORIAL2 Pipeline](https://www.quantopian.com/tutorials/pipeline) の、[Factors](https://www.quantopian.com/tutorials/pipeline#lesson3)をベースに、今回の勉強会に沿った形で展開しています。


## はじめに
デフォルトの状態（空の状態）のパイプラインを実行すると、

* 「Quantopianで利用可能な全銘柄」,「取引時刻」をインデックスとし
* 列データを持たない（0 Columns）

のDataframeを出力することを確認しました。つまりパイプラインを構築するということは、出力されるDataframeの列データを定義することを意味します。

```
F(asset, timestamp) -> float
```
出力されるDataframeのColumnにセットされる数値データのことを、Quantopianでは「ファクター」と呼びます。この呼び方は、今日のテーマである「ファクターモデル」を意識してつけているのではないかと思います。

ファクターを計算するために、Quantopianではよく使われるものを「組み込み（Built-in）」として提供しています。[built-in Factors](https://www.quantopian.com/help#built-in-factors).

まずはパイプラインの作成例として、built-in を利用してみます。ここで作成するファクターは、テクニカル分析に用いられるファクターです。


In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# ファクターの元データとなるデータソースをインポート（米国株価）
from quantopian.pipeline.data.builtin import USEquityPricing

# 組み込みファクターから、単純移動平均をインポート
from quantopian.pipeline.factors import SimpleMovingAverage

### ファクターをパイプラインに追加する

空のパイプラインにファクターを追加します。

SimpleMovingAverageを始めとする、全てのFactorをパイプラインで利用するためには、

* inputs = 計算元となる入力データ
* window_length = ウィンドウ期間

を指定する必要があります。詳しくは、[ここ](https://www.quantopian.com/help#building-computations)を参照のこと。

そして次に、作成したファクターを、パイプラインのColumnにセットします。


以上の工程を表すコードを、先程のmake_pipeline()に対し、以下のように改造します。

In [ ]:
def make_pipeline():
    
    # 米国株データベースから終値を取得して、10日間移動平均を計算する
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    # 終値
    latest_close = USEquityPricing.close.latest
    
    return Pipeline(
        # ファクターとして、10日間移動平均と、終値を追加
        columns={
            '10_day_mean_close': mean_close_10,
            'latest_close_price': latest_close
        }
    )

### 【Memo】 make_pipelineでfactorを定義する際の個人的な所感

パイプラインの中でファクターを定義するのは、Excelのセル内に相対セル指定を使って計算式を書いて、縦方向に計算式をコピペしているいく感覚に似ている？

### run_pipelineを実行する

先程と同様に、run_pipelineを実行します。
Dataframeに、factorがセットされたことを確認します。

In [ ]:
result = run_pipeline(make_pipeline(), '2018-11-01', '2018-11-05')
result

### マルチインデクス

パイプラインの出力であるDataframeは、「営業日」「銘柄コード」をインデクスとしたDataframeとなっています。Pandasにおいて、複数のインデクスを持つDataframeをマルチインデクスと呼びます。複数期間のデータを持つパイプラインから、特定営業日をキーとしてデータを抜き出したい場合には、以下のようにします。

In [ ]:
result = run_pipeline(make_pipeline(), '2018-11-01', '2018-11-05')
result.loc[("2018-11-01")]